## Save HTML Files for GameIDs

In [94]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import codecs
import os
import time

baseurl = 'https://fibalivestats.dcd.shared.geniussports.com/u/IBBA'  
start_id = 2160600
end_id = 2160820

for g_id in range(start_id, end_id + 1):     
    
    url = "{}/{}/pbp.html".format(baseurl, g_id)
    resp = requests.get(url)
    if resp.status_code == 200:   
            driver = webdriver.Chrome()
            driver.get(url)
            path = "C:\\bball data\\Games HTML\\{}.html".format(g_id)
            with open(path, "w", encoding='utf-8') as f:
                f.write(driver.page_source)
            driver.quit()
            time.sleep(5)
    else:
        pass


## Scrape PBP HTML File to Dataframe

In [8]:
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

start_id = int(input ('Please enter GameID to start with'))
end_id = int(input ('Please enter GameID to end with'))
data = []

for g_id in range(start_id, end_id + 1):     

    file_path = f"C:\\bball data\\Games HTML\\{g_id}.html"  
    with open(file_path, encoding='utf-8') as f:
        soup = BeautifulSoup(f, "html.parser")

    t1 = soup.find('span', {'class': 'id_aj_1_name'}).text
    t2 = soup.find('span', {'class': 'id_aj_2_name'}).text 

    pbp_div = soup.find("div", id="aj_pbp")

    for div in pbp_div.find_all('div', {'class': 'pbpa'}):
            row = {}

            row['GameID'] = g_id
            row['Time'] = div['id']       
            row['Period'] = div['class'][3][4:8]
            row['Type'] = div['class'][-1]
            row['Specific_Type'] = div['class'][-2]

            ##row['Full_Type'] = (row['Short_Type'],' ',row['Specific_Type'])
            row['Team'] = div['class'][1][3:5]
            if row['Team'] == 't1':
                row['Team'] = t1
            elif row['Team'] == 't2':
                row['Team'] = t2
            else:
                row['Team'] = ''

            if div.find('strong') != None:
                P, PN = div.find('strong').text.split(',')
                row['Player'] = P
                row['JerseyNumber'] = int(PN)
            else:
                row['Player'] = ''
                row['JerseyNumber'] = ''            

            row['Score'] = div.find('span', {'class': 'pbpsc'})
            if row['Score'] != None:
                row['Score'] = div.find('span', {'class': 'pbpsc'}).text
                s1, s2 = div.find('span', {'class': 'pbpsc'}).text.split('-')
                row['Abs_Diff'] = abs((int(s1))-(int(s2)))
                row['Diff'] = (int(s1))-(int(s2)) 
            if  ('2pt' in row['Type']) or ('3pt' in row['Type']) or ('freethrow' in row['Type']):
                if row['Score'] != None:
                    row['Type'] = row['Type']+' Made'
                else:
                    row['Type'] = row['Type']+' Missed'
            data.append(row)
    
    df = pd.DataFrame(data)
    df['Score'].fillna(method='backfill', inplace=True)
    df['Abs_Diff'].fillna(method='backfill', inplace=True)
    df['Diff'].fillna(method='backfill', inplace=True)
    df['Type'] = df['Type'].str.title()
    df['Specific_Type'] = df['Specific_Type'].str.title()
    df = df.sort_index(ascending=False)
    print('GameID ',g_id,'scraped successfully - ',(len(df[df['GameID']==g_id])),' Rows')   


Please endet GameID to start with 2160694
Please endet GameID to end with 2160696


GameID  2160694 scraped successfully -  512  Rows
GameID  2160695 scraped successfully -  530  Rows
GameID  2160696 scraped successfully -  514  Rows


In [7]:
df

,GameID,Time,Period,Type,Specific_Type,Team,Player,JerseyNumber,Score,Abs_Diff,Diff
1041,2160695,10:00,1,Game,Start,,,,0-0,0.0,0.0
1040,2160695,10:00,1,Period,Start,,,,0-0,0.0,0.0
1039,2160695,10:00,1,Jumpball,Startperiod,,,,0-0,0.0,0.0
1038,2160695,09:59,1,Jumpball,Won,מכבי אפי קפיטל ראשלצ,ר. בוקלס,5,0-0,0.0,0.0
1037,2160695,09:59,1,Jumpball,Lost,מכבי עירוני רעננה,א. מורס,0,0-0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
4,2160694,00:03,4,Substitution,In,אליצור שומרון,י. מיכאלי,15,78-77,1.0,1.0
3,2160694,00:03,4,Substitution,Out,אליצור שומרון,ג. שטרנברג,7,78-77,1.0,1.0
2,2160694,00:02,4,2Pt Missed,Layup,מכבי נקסט אורבן חיפה,א. גילדר,3,78-77,1.0,1.0
1,2160694,00:00,4,Period,End,,,,78-77,1.0,1.0


## Scrape schedule into Dataframe